In [1]:
import typesense
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']

In [3]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

In [21]:
search_parameters = {
    'q'         : 'sasuke uchiha',
    'query_by'  : 'name',
#   'filter_by' : 'num_employees:>100',
#   'sort_by'   : 'num_employees:desc'
}

search_hits = client.collections['entity'].documents.search(search_parameters)
search_hits

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'age': 59,
    'dateOfBirth': '1964-09-25',
    'description': "[Website](https://www.manbow.com/) | [Profile](http://office-anemone.com/mt-search.cgi?IncludeBlogs=4,5&tag=井上喜久子) | [Twitter](https://twitter.com/atmanbow_staff)\n\nMother of voice actress [Honoka Inoue](https://anilist.co/staff/129363/Honoka-Inoue). She has held a long-running gag of being 'eternally 17 years old'.",
    'gender': 'Female',
    'homeTown': 'Yokosuka, Kanagawa Prefecture, Japan',
    'id': 'People/95195',
    'image_url': 'https://s4.anilist.co/file/anilistcdn/staff/medium/n95195-nLvtZl5sCK0D.png',
    'language': 'Japanese',
    'name': 'Kikuko Inoue',
    'name_first': 'Kikuko',
    'primaryOccupations': ['Voice Actor'],
    'synonyms': ['Kikuko', 'Inoue', '井上喜久子', '井之上喜久子'],
    'type': ['People'],
    'url': 'https://anilist.co/staff/95195',
    'yearsActive': [1988]},
   'highlight': {'name': {'matched_tokens': ['Kikuko', 'Inoue'],
     'snipp

In [13]:
client.collections['webpage'].retrieve()

{'created_at': 1706362006,
 'default_sorting_field': '',
 'enable_nested_fields': False,
 'fields': [{'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'title',
   'optional': True,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'url',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'aka',
   'optional': True,
   'sort': False,
   'type': 'string[]'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'site',
   'optional': True,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'site_type',
   'optional': True,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'entity_type',
   'optional': True,
   'sort'